<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# 用于大语言模型（LLM）校准的直接偏好优化（DPO）（从零开始） 

- 本代码笔记本从零开始实现了直接偏好优化（DPO），并将其应用于一个大语言模型（LLM），以提升该模型生成更符合用户偏好的回复的能力。 

In [1]:
# !pip install -r https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/requirements.txt

In [2]:
from importlib.metadata import version

pkgs = [
    "tiktoken",    # Tokenizer
    "torch",       # Deep learning library
]
for p in pkgs:
    print(f"{p} version: {version(p)}")

tiktoken version: 0.8.0
torch version: 2.5.1


&nbsp;
# 1) A brief introduction to DPO

- 在论文 [Direct Preference Optimization: Your Language Model is Secretly a Reward Model](https://arxiv.org/abs/2305.18290) 当中，是用于微调大语言模型（LLM）的基于人类反馈的强化学习（RLHF）的一种替代方法。
- 直接偏好优化（DPO）可用于微调（或校准）模型，以生成更符合用户期望和指令的回复。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/dpo/1.webp" width=700px>

- 在指令微调中，我们训练大语言模型（LLM），使其在给定提示的情况下生成正确答案。
- 然而，在实际中，有多种方式可以给出正确答案，并且正确答案在风格上可能有所不同；例如，当要求大语言模型（LLM）在购买笔记本电脑时给出建议时，会有技术风格的回复和更便于用户理解的回复这两种情况，如下图所示。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/dpo/2.webp" width=700px>

- 基于人类反馈的强化学习（RLHF）和直接偏好优化（DPO）是可用于教导大语言模型（LLM）偏好一种答案风格而非另一种的方法，也就是说，使其更好地符合用户偏好。
- 基于人类反馈的强化学习（RLHF）过程需要训练一个单独的奖励模型，其概述如下。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/dpo/4.webp" width=700px> 

- 与基于人类反馈的强化学习（RLHF）相比，直接偏好优化（DPO）旨在通过直接针对用户偏好来优化模型，从而简化流程，无需进行复杂的奖励建模和策略优化。
- 换句话说，直接偏好优化（DPO）专注于直接优化模型的输出，使其符合人类偏好或特定目标。
- 下面展示的是直接偏好优化（DPO）工作原理的主要思路概述。 

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/dpo/5.webp?123" width=800px>

- 实现直接偏好优化（DPO）损失的具体公式如下所示；在本代码笔记本后面我们用 Python 实现它时，会再次提及这个公式。 

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/dpo/3.webp?123" width=800px>

- 在上述公式中：
    - “期望值” $\mathbb{E}$ 是统计学术语，表示随机变量（括号内的表达式）的平均值或均值；对 $-\mathbb{E}$ 进行优化能使模型更好地符合用户偏好。
    - $\pi_{\theta}$ 变量即所谓的策略（这是一个从强化学习中借用的术语），代表我们想要优化的大语言模型（LLM）；$\pi_{ref}$ 是一个参考大语言模型，通常是优化前的原始大语言模型（在训练开始时，$\pi_{\theta}$ 和 $\pi_{ref}$ 通常是相同的）。
    - $\beta$ 是一个超参数，用于控制 $\pi_{\theta}$ 与参考模型之间的差异；增大 $\beta$ 会增加 $\pi_{\theta}$ 和 $\pi_{ref}$ 之间在对数概率方面的差异对整体损失函数的影响，从而增大两个模型之间的差异。
    - 逻辑 sigmoid 函数 $\sigma(\centerdot)$ 将偏好和被拒绝回复的对数几率（逻辑 sigmoid 函数内的项）转换为一个概率分数。
- 为了避免在本代码笔记本中进行过于详细的讨论而使其内容臃肿，我未来可能会单独撰写一篇独立的文章，对这些概念进行更详细的阐述。
- 同时，如果你有兴趣比较基于人类反馈的强化学习（RLHF）和直接偏好优化（DPO），请查看我文章 [大语言模型预训练和评估奖励模型的技巧](https://magazine.sebastianraschka.com/p/tips-for-llm-pretraining-and-evaluating-rms)中 [2.2. 基于人类反馈的强化学习（RLHF）与直接偏好优化（DPO）](https://magazine.sebastianraschka.com/i/142924793/rlhf-vs-direct-preference-optimization-dpo) 这一部分。 

&nbsp;
# 2) 为直接偏好优化（DPO）准备一个偏好数据集。 

- 让我们从加载和准备数据集开始，在我们再次探讨直接偏好优化（DPO）损失公式之前，这可能已经能解答你心中的许多疑问了。
- 在这里，我们使用的数据集包含了对指令提示的更礼貌和不太礼貌的回复（具体示例将在下一部分展示）。
- 该数据集是通过 [create-preference-data-ollama.ipynb](create-preference-data-ollama.ipynb) 这个笔记本生成的。  


&nbsp;
## 2.1) 加载偏好数据集

- 该数据集是一个包含1100条记录的JSON文件： 

In [3]:
import json
import os
import urllib


def download_and_load_file(file_path, url):

    if not os.path.exists(file_path):
        with urllib.request.urlopen(url) as response:
            text_data = response.read().decode("utf-8")
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(text_data)
    else:
        with open(file_path, "r", encoding="utf-8") as file:
            text_data = file.read()

    with open(file_path, "r", encoding="utf-8") as file:
        data = json.load(file)

    return data


file_path = "instruction-data-with-preference.json"
url = (
    "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch"
    "/main/ch07/04_preference-tuning-with-dpo/instruction-data-with-preference.json"
)

data = download_and_load_file(file_path, url)
print("Number of entries:", len(data))

Number of entries: 1100


- 让我们来看一下两个示例记录： 

In [4]:
import pprint

pprint.pp(data[50])

{'instruction': 'Identify the correct spelling of the following word.',
 'input': 'Ocassion',
 'output': "The correct spelling is 'Occasion.'",
 'rejected': "The correct spelling is obviously 'Occasion.'",
 'chosen': "The correct spelling is 'Occasion.'"}


In [5]:
pprint.pp(data[999])

{'instruction': "What is an antonym of 'complicated'?",
 'input': '',
 'output': "An antonym of 'complicated' is 'simple'.",
 'chosen': "A suitable antonym for 'complicated' would be 'simple'.",
 'rejected': "An antonym of 'complicated' is 'simple'."}


```
# 这是按代码格式进行的排版
```

- 正如我们在上面所看到的，该数据集由5个键组成：
    - `'instruction'`（指令）和 `'input'`（输入），它们被用作大语言模型（LLM）的输入。
    - `'output'`（输出）包含了在第7章中通过指令微调步骤对模型进行训练时所依据的回复内容。
    - `'chosen'`（选中的）和 `'rejected'`（被拒绝的）条目是我们用于直接偏好优化（DPO）的内容；在这里，`'chosen'` 是更受偏好的回复，而 `'rejected'` 是不太受偏好的回复。
- 目标是让模型遵循被选中的回复风格，而不是被拒绝的回复风格。 

- 下面是一个实用函数，它会像第7章（[../01_main-chapter-code/ch07.ipynb](../01_main-chapter-code/ch07.ipynb)）那样，采用阿尔帕卡（Alpaca）提示风格来格式化模型输入： 

In [6]:
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""

    return instruction_text + input_text

In [7]:
model_input = format_input(data[50])
print(model_input)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Identify the correct spelling of the following word.

### Input:
Ocassion


- 同样地，我们可以使用 Alpaca 提示风格来格式化被选中和被拒绝的回复：

In [8]:
desired_response = f"### Response:\n{data[50]['chosen']}"
print(desired_response)

### Response:
The correct spelling is 'Occasion.'


In [9]:
possible_response = f"### Response:\n{data[50]['rejected']}"
print(possible_response)

### Response:
The correct spelling is obviously 'Occasion.'


&nbsp;
## 2.2) 创建训练集、验证集和测试集的划分。 

- 接下来，我们将数据集划分为三个子集，分别是 85% 的训练数据、5% 的验证数据和 10% 的测试数据： 

In [10]:
train_portion = int(len(data) * 0.85)  # 85% for training
test_portion = int(len(data) * 0.1)    # 10% for testing
val_portion = len(data) - train_portion - test_portion  # Remaining 5% for validation

train_data = data[:train_portion]
test_data = data[train_portion:train_portion + test_portion]
val_data = data[train_portion + test_portion:]

In [11]:
print("Training set length:", len(train_data))
print("Validation set length:", len(val_data))
print("Test set length:", len(test_data))

Training set length: 935
Validation set length: 55
Test set length: 110


&nbsp;
## 2.3) 开发一个 `PreferenceDataset` 类以及批量处理函数。 

- 在本节中，我们为直接偏好优化（DPO）重写第 7 章（[../01_main-chapter-code/ch07.ipynb](../01_main-chapter-code/ch07.ipynb)）中的 `InstructionDataset` 类。
- 这意味着我们不再关注单个输出序列（回复），而是修改数据集类，使其返回成对的回复，其中一个（“选中的”）比另一个（“被拒绝的”）更受偏好。
- 总体而言，`PreferenceDataset` 与第 7 章中使用的 `InstructionDataset` 几乎相同： 

In [12]:
import torch
from torch.utils.data import Dataset


class PreferenceDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data

        # Pre-tokenize texts
        self.encoded_texts = []
        for entry in data:
            prompt = format_input(entry)
            rejected_response = entry["rejected"]
            chosen_response = entry["chosen"]

            prompt_tokens = tokenizer.encode(prompt)
            chosen_full_text = f"{prompt}\n\n### Response:\n{chosen_response}"
            rejected_full_text = f"{prompt}\n\n### Response:\n{rejected_response}"
            chosen_full_tokens = tokenizer.encode(chosen_full_text)
            rejected_full_tokens = tokenizer.encode(rejected_full_text)

            self.encoded_texts.append({
                "prompt": prompt_tokens,
                "chosen": chosen_full_tokens,
                "rejected": rejected_full_tokens,
            })

    def __getitem__(self, index):
        return self.encoded_texts[index]

    def __len__(self):
        return len(self.data)


- 除了更新后的 `PreferenceDataset` 类，我们还需要一个更新后的批量整理函数，用于将每个批次中的序列填充到相同的长度，这样我们就可以将它们组装成批次。
- 我在下面的代码中添加了注释来说明这个过程；不过，通过查看下面的示例输入和输出，可能最容易理解它是如何工作的：

In [13]:
def custom_collate_fn(
    batch,
    pad_token_id=50256,
    allowed_max_length=None,
    mask_prompt_tokens=True,
    device="cpu"
):
    # Initialize lists to hold batch data
    batch_data = {
        "prompt": [],
        "chosen": [],
        "rejected": [],
        "rejected_mask": [],
        "chosen_mask": []

    }

    # Determine the longest sequence to set a common padding length
    max_length_common = 0
    if batch:
        for key in ["chosen", "rejected"]:
            current_max = max(len(item[key])+1 for item in batch)
            max_length_common = max(max_length_common, current_max)

    # Process each item in the batch
    for item in batch:
        prompt = torch.tensor(item["prompt"])
        batch_data["prompt"].append(prompt)

        for key in ["chosen", "rejected"]:
            # Adjust padding according to the common maximum length
            sequence = item[key]
            padded = sequence + [pad_token_id] * (max_length_common - len(sequence))
            mask = torch.ones(len(padded)).bool()

            # Set mask for all padding tokens to False
            mask[len(sequence):] = False

            # Set mask for all input tokens to False
            # +2 sets the 2 newline ("\n") tokens before "### Response" to False
            if mask_prompt_tokens:
                mask[:prompt.shape[0]+2] = False

            batch_data[key].append(torch.tensor(padded))
            batch_data[f"{key}_mask"].append(mask)

    # Final processing
    for key in ["chosen", "rejected", "chosen_mask", "rejected_mask"]:
        # Stack all sequences into a tensor for the given key
        tensor_stack = torch.stack(batch_data[key])

        # Optionally truncate to maximum sequence length
        if allowed_max_length is not None:
            tensor_stack = tensor_stack[:, :allowed_max_length]

        # Move to the specified device
        batch_data[key] = tensor_stack.to(device)

    return batch_data

- 在我们开始使用自定义的整理函数（custom collate function）之前，让我们先创建一个该函数的版本，其中预先填充了它的一些函数参数： 

In [14]:
from functools import partial

device = torch.device(
    "cuda" if torch.cuda.is_available() 
    else "mps" if torch.backends.mps.is_available() 
    else "cpu"
)
print("Device:", device)

customized_collate_fn = partial(
    custom_collate_fn,
    device=device,            # Put the data directly on a GPU if available
    mask_prompt_tokens=True,  # This is optional
    allowed_max_length=1024   # The supported context length of the model
)

Device: mps


- 现在，让我们看看 `customized_collate_fn` 是如何运行的，并将其应用到我们偏好数据集中的一些样本数据上；为此，我们取前两条记录： 

In [15]:
example_data = data[:2]

for i in example_data:
    print()
    pprint.pp(i)


{'instruction': 'Evaluate the following phrase by transforming it into the '
                'spelling given.',
 'input': 'freind --> friend',
 'output': 'The spelling of the given phrase "freind" is incorrect, the '
           'correct spelling is "friend".',
 'rejected': 'The spelling of the given phrase "freind" is flat out wrong, get '
             'it together, the correct spelling is "friend".',
 'chosen': 'The spelling of the given phrase "freind" is incorrect, the '
           'correct spelling is "friend".'}

{'instruction': 'Edit the following sentence for grammar.',
 'input': 'He go to the park every day.',
 'output': 'He goes to the park every day.',
 'rejected': 'He goes to the stupid park every single day.',
 'chosen': 'He goes to the park every day.'}


- 接下来，让我们实例化一个 `example_dataset`，并使用 PyTorch 的 `DataLoader` 来创建一个 `example_dataloader`，它模拟了我们稍后将用于模型训练的数据加载器： 

In [16]:
import tiktoken
from torch.utils.data import DataLoader


tokenizer = tiktoken.get_encoding("gpt2")

example_dataset = PreferenceDataset(example_data, tokenizer)

example_dataloader = DataLoader(
    example_dataset,
    batch_size=2,
    collate_fn=customized_collate_fn,
    shuffle=False
)

- 该数据集具有以下的 key：

In [17]:
for batch in example_dataloader:
    break

print("batch.keys:", batch.keys())

batch.keys: dict_keys(['prompt', 'chosen', 'rejected', 'rejected_mask', 'chosen_mask'])


- 这些提示（prompts）是一个张量列表，其中每个张量包含给定示例的标记 ID（token IDs）；由于我们选择的批量大小为 2，所以这里有两个标记 ID 张量列表： 

In [18]:
batch["prompt"]

[tensor([21106,   318,   281, 12064,   326,  8477,   257,  4876,    13, 19430,
           257,  2882,   326, 20431, 32543,   262,  2581,    13,   198,   198,
         21017, 46486,    25,   198,    36,  2100,  4985,   262,  1708,  9546,
           416, 25449,   340,   656,   262, 24993,  1813,    13,   198,   198,
         21017, 23412,    25,   198, 19503,   521, 14610,  1545]),
 tensor([21106,   318,   281, 12064,   326,  8477,   257,  4876,    13, 19430,
           257,  2882,   326, 20431, 32543,   262,  2581,    13,   198,   198,
         21017, 46486,    25,   198, 18378,   262,  1708,  6827,   329, 23491,
            13,   198,   198, 21017, 23412,    25,   198,  1544,   467,   284,
           262,  3952,   790,  1110,    13])]

- 我们在训练时实际上并不需要这些回复（responses）；在训练过程中，我们需要输入到模型中的是“chosen”和“rejected”条目。
- “chosen”和“rejected”回复条目都进行了填充（padded），以便我们可以将它们堆叠为张量（tensors）；与提示（prompts）类似，这些回复文本被编码为标记 ID（token IDs）： 

In [19]:
batch["chosen"]

tensor([[21106,   318,   281, 12064,   326,  8477,   257,  4876,    13, 19430,
           257,  2882,   326, 20431, 32543,   262,  2581,    13,   198,   198,
         21017, 46486,    25,   198,    36,  2100,  4985,   262,  1708,  9546,
           416, 25449,   340,   656,   262, 24993,  1813,    13,   198,   198,
         21017, 23412,    25,   198, 19503,   521, 14610,  1545,   198,   198,
         21017, 18261,    25,   198,   464, 24993,   286,   262,  1813,  9546,
           366, 19503,   521,     1,   318, 11491,    11,   262,  3376, 24993,
           318,   366,  6726,  1911, 50256, 50256, 50256, 50256, 50256, 50256,
         50256],
        [21106,   318,   281, 12064,   326,  8477,   257,  4876,    13, 19430,
           257,  2882,   326, 20431, 32543,   262,  2581,    13,   198,   198,
         21017, 46486,    25,   198, 18378,   262,  1708,  6827,   329, 23491,
            13,   198,   198, 21017, 23412,    25,   198,  1544,   467,   284,
           262,  3952,   790,  1110

- 上面的标记 ID（token IDs）代表了模型的输入，但以这种格式呈现时，我们人类很难理解它们的含义。
- 所以，让我们实现一个小的实用函数，将它们转换回文本形式，这样我们就能更轻松地检查和理解这些输入了： 

In [20]:
def decode_tokens_from_batch(token_ids, tokenizer):
    ids_in_python_list = token_ids.flatten().tolist()
    return tokenizer.decode(ids_in_python_list)

- 让我们将 `decode_tokens_from_batch` 实用函数应用于批次中的第一个提示条目： 

In [21]:
text = decode_tokens_from_batch(
    token_ids=batch["prompt"][0],  # [0] for the first entry in the batch
    tokenizer=tokenizer,
)
print(text)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Evaluate the following phrase by transforming it into the spelling given.

### Input:
freind --> friend


- 正如我们在上面看到的，提示（prompt）的格式是正确的；现在让我们对 “chosen” 回复执行相同的操作： 

In [22]:
text = decode_tokens_from_batch(
    token_ids=batch["chosen"][0],
    tokenizer=tokenizer,
)
print(text)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Evaluate the following phrase by transforming it into the spelling given.

### Input:
freind --> friend

### Response:
The spelling of the given phrase "freind" is incorrect, the correct spelling is "friend".<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>


- 正如我们在上面所看到的，与指令微调（instruction finetuning）类似，在训练期间传递给模型的回复也包含了输入提示（input prompt）。
- 另请注意，我们将 `<|endoftext|>` 标记用作填充标记（padding tokens），这是必要的，这样我们就可以将回复扩展到相似的长度，以便将它们堆叠成一个批次。
- 不用担心；稍后在计算损失（loss）时，`<|endoftext|>` 标记将被忽略，因此它们不会影响训练结果。
- 现在让我们也检查一下相应的被拒绝的回复： 

In [23]:
text = decode_tokens_from_batch(
    token_ids=batch["rejected"][0],
    tokenizer=tokenizer,
)
print(text)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Evaluate the following phrase by transforming it into the spelling given.

### Input:
freind --> friend

### Response:
The spelling of the given phrase "freind" is flat out wrong, get it together, the correct spelling is "friend".<|endoftext|>


- 在这种情况下，正如我们在上面看到的，被拒绝的回复是被选中回复的一个更不礼貌的版本（我们不希望模型生成不礼貌的回复）。
- 最后，让我们谈谈数据掩码（data masks）：如果你仔细看一下我们上面实现的自定义整理函数（custom collate function），我们为每个数据集条目创建了一个“chosen_mask”和一个“rejected_mask”。
- 掩码与回复条目具有相同的形状，如下所示为“chosen”条目的情况： 

In [24]:
print("chosen inputs:", batch["chosen"][0].shape)
print("chosen mask:  ", batch["chosen_mask"][0].shape)

chosen inputs: torch.Size([81])
chosen mask:   torch.Size([81])


- The contents of these masks are boolean (`True` and `False`) values:

In [25]:
batch["chosen_mask"][0]

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True, False, False, False, False, False, False,
        False], device='mps:0')

- `True` 值表示对应实际回复的标记 ID（token IDs）。
- `False` 标记对应于要么是提示标记（prompt tokens）（如果我们在 `customized_collate_fn` 函数中设置 `mask_prompt_tokens=True`，我们之前就是这么做的）要么是填充标记（padding tokens）的标记 ID。
- 因此，我们可以将掩码用作选择掩码，仅选择对应回复的标记 ID，也就是去除所有提示和填充标记，如下所示： 

In [26]:
text = decode_tokens_from_batch(
    token_ids=batch["chosen"][0][batch["chosen_mask"][0]],
    tokenizer=tokenizer,
)
print(text)

### Response:
The spelling of the given phrase "freind" is incorrect, the correct spelling is "friend".


In [27]:
text = decode_tokens_from_batch(
    token_ids=batch["rejected"][0][batch["rejected_mask"][0]],
    tokenizer=tokenizer,
)
print(text)

### Response:
The spelling of the given phrase "freind" is flat out wrong, get it together, the correct spelling is "friend".


- 稍后在计算 DPO 损失时，我们将利用这个掩码来忽略提示标记和填充标记。 

&nbsp;
## 2.4) 创建训练集、验证集和测试集的数据加载器

- 上述内容我们使用了偏好数据集里的一个小型示例子集进行演示。
- 现在，让我们来创建实际的训练集、验证集和测试集的数据加载器。
- 这个过程与预训练和指令微调章节中创建数据加载器的过程相同，因此应该一目了然。 

In [28]:
from torch.utils.data import DataLoader


num_workers = 0
batch_size = 8

torch.manual_seed(123)

train_dataset = PreferenceDataset(train_data, tokenizer)
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    collate_fn=customized_collate_fn,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers
)

In [29]:
val_dataset = PreferenceDataset(val_data, tokenizer)
val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    collate_fn=customized_collate_fn,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers
)

test_dataset = PreferenceDataset(test_data, tokenizer)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    collate_fn=customized_collate_fn,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers
)

- Let's iterate through the data loader and take a look at the dataset shapes:

In [30]:
print("Train loader:")
for batch in train_loader:
    print(
        batch["chosen"].shape,
        batch["rejected"].shape,
    )

Train loader:
torch.Size([8, 77]) torch.Size([8, 77])
torch.Size([8, 81]) torch.Size([8, 81])
torch.Size([8, 94]) torch.Size([8, 94])
torch.Size([8, 75]) torch.Size([8, 75])
torch.Size([8, 75]) torch.Size([8, 75])
torch.Size([8, 76]) torch.Size([8, 76])
torch.Size([8, 99]) torch.Size([8, 99])
torch.Size([8, 71]) torch.Size([8, 71])
torch.Size([8, 67]) torch.Size([8, 67])
torch.Size([8, 88]) torch.Size([8, 88])
torch.Size([8, 65]) torch.Size([8, 65])
torch.Size([8, 79]) torch.Size([8, 79])
torch.Size([8, 80]) torch.Size([8, 80])
torch.Size([8, 97]) torch.Size([8, 97])
torch.Size([8, 71]) torch.Size([8, 71])
torch.Size([8, 89]) torch.Size([8, 89])
torch.Size([8, 75]) torch.Size([8, 75])
torch.Size([8, 69]) torch.Size([8, 69])
torch.Size([8, 84]) torch.Size([8, 84])
torch.Size([8, 79]) torch.Size([8, 79])
torch.Size([8, 101]) torch.Size([8, 101])
torch.Size([8, 87]) torch.Size([8, 87])
torch.Size([8, 73]) torch.Size([8, 73])
torch.Size([8, 69]) torch.Size([8, 69])
torch.Size([8, 80]) torc

- 每一行展示了每个批次中“chosen”和“rejected”条目的形状。
- 由于我们是逐批次进行填充操作的，所以每一行的形状都不同。
- 这是出于效率考虑，因为将全量数据集中的所有样本都填充到最长样本的长度会很低效。 

&nbsp;
# 3) 加载一个经过微调的大语言模型（LLM）以进行直接偏好优化（DPO）对齐

- 大语言模型（LLM）对齐步骤，如基于人类反馈的强化学习（RLHF）或直接偏好优化（DPO），都假定我们已经有了一个经过指令微调的模型。
- 本节包含最少的代码，用于加载在第 7 章中经过指令微调并保存的模型（通过 [../01_main-chapter-code/ch07.ipynb](../01_main-chapter-code/ch07.ipynb)）。
- 在继续操作之前，请确保你先运行第 7 章的代码以创建经过指令微调的模型。
- 下面的代码将把经过指令微调的模型复制到当前目录： 

In [31]:
from pathlib import Path
import shutil


finetuned_model_path = Path("gpt2-medium355M-sft.pth")
if not finetuned_model_path.exists():

    # Try finding the model checkpoint locally:
    relative_path = Path("..") / "01_main-chapter-code" / finetuned_model_path
    if relative_path.exists():
        shutil.copy(relative_path, ".")

    # If this notebook is run on Google Colab, get it from a Google Drive folder
    elif "COLAB_GPU" in os.environ or "COLAB_TPU_ADDR" in os.environ:
        from google.colab import drive
        drive.mount("/content/drive")
        google_drive_path = "/content/drive/My Drive/Books/LLMs-From-Scratch/ch07/colab/gpt2-medium355M-sft.pth"  # Readers need to adjust this path
        shutil.copy(google_drive_path, ".")

    else:
        print(
            f"Could not find '{finetuned_model_path}'.\n"
            "Run the `ch07.ipynb` notebook to finetune and save the finetuned model."
        )

- 接下来，我们复用前几章的基本配置来加载模型权重：

In [32]:
from previous_chapters import GPTModel


BASE_CONFIG = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "drop_rate": 0.0,        # Dropout rate
    "qkv_bias": True         # Query-key-value bias
}

model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

CHOOSE_MODEL = "gpt2-medium (355M)"

BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

model = GPTModel(BASE_CONFIG)

In [33]:
model.load_state_dict(
    torch.load(
        "gpt2-medium355M-sft.pth",
        map_location=torch.device("cpu"),
        weights_only=True
    )
)
model.eval();

- 在使用直接偏好优化（DPO）方法对加载的模型进行训练之前，让我们通过在一些样本数据上进行测试，来确保经过微调的模型已正确保存和加载： 

In [34]:
prompt = """Below is an instruction that describes a task. Write a response
that appropriately completes the request.

### Instruction:
Convert the active sentence to passive: 'The chef cooks the meal every day.'
"""

In [35]:
from previous_chapters import (
    generate,
    text_to_token_ids,
    token_ids_to_text
)

torch.manual_seed(123)

token_ids = generate(
    model=model,
    idx=text_to_token_ids(prompt, tokenizer),
    max_new_tokens=35,
    context_size=BASE_CONFIG["context_length"],
    eos_id=50256
)

response = token_ids_to_text(token_ids, tokenizer)
print(response)

Below is an instruction that describes a task. Write a response
that appropriately completes the request.

### Instruction:
Convert the active sentence to passive: 'The chef cooks the meal every day.'

### Response:
The meal is cooked every day by the chef.


- 正如我们在上面看到的，模型给出了合理且正确的回复。
- 正如第 7 章所解释的，在实际应用中，我们会对回复进行清理，只返回去除提示和提示风格后的回复文本（例如，类似于你熟悉的 ChatGPT 的做法）。 

In [36]:
def extract_response(response_text, input_text):
    return response_text[len(input_text):].replace("### Response:", "").strip()

response = extract_response(response, prompt)
print(response)

The meal is cooked every day by the chef.


- 现在，我们几乎准备好进入直接偏好优化（DPO）部分了。
- 正如本笔记开头所提到的，DPO 会用到两个大语言模型（LLM）：一个是策略模型（我们想要优化的 LLM），另一个是参考模型（我们保持不变的原始模型）。
- 下面，我们将 `model` 重命名为 `policy_model`，并实例化该模型的第二个实例，我们将其称为 `reference_model`。 

In [37]:
policy_model = model

reference_model = GPTModel(BASE_CONFIG)
reference_model.load_state_dict(
    torch.load(
        "gpt2-medium355M-sft.pth",
        map_location=torch.device("cpu"),
        weights_only=True
    )
)
reference_model.eval()

policy_model.to(device)
reference_model.to(device);

In [38]:
print(device)

mps


&nbsp;
# 4) 编写 DPO Loss Function 的代码

- 在前几节中我们处理好了模型加载和数据集准备工作之后，现在我们可以进入有趣的部分，为直接偏好优化（DPO）损失函数编写代码了。
- 请注意，下面的直接偏好优化（DPO）损失函数代码是基于 [《直接偏好优化：你的语言模型其实是一个奖励模型》](https://arxiv.org/abs/2305.18290) 这篇论文中提出的方法。
- 作为参考，核心的直接偏好优化（DPO）公式再次展示如下： 

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/dpo/3.webp?123" width=800px>

- 在上述公式中：
  - “期望值” $\mathbb{E}$ 是统计学领域的术语，表示随机变量（括号内的表达式）的平均值或均值；对 $-\mathbb{E}$ 进行优化能使模型更好地符合用户偏好。
  - $\pi_{\theta}$ 变量就是所谓的策略（这一术语借自强化学习领域），代表我们想要优化的大语言模型（LLM）；$\pi_{ref}$ 是一个参考大语言模型，通常是优化前的原始大语言模型（在训练开始时，$\pi_{\theta}$ 和 $\pi_{ref}$ 通常是相同的）。
  - $\beta$ 是一个超参数，用于控制 $\pi_{\theta}$ 与参考模型之间的差异；增大 $\beta$ 会增加 $\pi_{\theta}$ 和 $\pi_{ref}$ 在对数概率方面的差异对整体损失函数的影响，从而增大这两个模型之间的差异。
  - 逻辑 sigmoid 函数 $\sigma(\centerdot)$ 将偏好和拒绝的响应的对数几率（逻辑 sigmoid 函数内的项）转换为概率分数。
- 在代码中，我们可以按如下方式实现直接偏好优化（DPO）损失函数： 

In [39]:
import torch.nn.functional as F

def compute_dpo_loss(
      model_chosen_logprobs,
      model_rejected_logprobs,
      reference_chosen_logprobs,
      reference_rejected_logprobs,
      beta=0.1,
    ):
    """Compute the DPO loss for a batch of policy and reference model log probabilities.

    Args:
        policy_chosen_logprobs: Log probabilities of the policy model for the chosen responses. Shape: (batch_size,)
        policy_rejected_logprobs: Log probabilities of the policy model for the rejected responses. Shape: (batch_size,)
        reference_chosen_logprobs: Log probabilities of the reference model for the chosen responses. Shape: (batch_size,)
        reference_rejected_logprobs: Log probabilities of the reference model for the rejected responses. Shape: (batch_size,)
        beta: Temperature parameter for the DPO loss; typically something in the range of 0.1 to 0.5. We ignore the reference model as beta -> 0.
        label_smoothing: conservativeness for DPO loss.

    Returns:
        A tuple of three tensors: (loss, chosen_rewards, rejected_rewards).
    """

    model_logratios = model_chosen_logprobs - model_rejected_logprobs
    reference_logratios = reference_chosen_logprobs - reference_rejected_logprobs
    logits = model_logratios - reference_logratios

    # DPO (Eq. 7 of https://arxiv.org/pdf/2305.18290.pdf)
    losses = -F.logsigmoid(beta * logits)

    # Optional values to track progress during training
    chosen_rewards = (model_chosen_logprobs - reference_chosen_logprobs).detach()
    rejected_rewards = (model_rejected_logprobs - reference_rejected_logprobs).detach()

    # .mean() to average over the samples in the batch
    return losses.mean(), chosen_rewards.mean(), rejected_rewards.mean()

- 如果你熟悉对数运算，请注意我们有通用的关系 $\log\left(\frac{a}{b}\right) = \log a - \log b$，我们在上面的代码中应用了这一关系。
- 记住这一点，让我们来逐步分析其中的一些步骤（我们稍后会使用一个单独的函数来计算 `logprobs`）。
- 让我们从以下几行代码开始：

    ```python
    model_logratios = model_chosen_logprobs - model_rejected_logprobs
    reference_logratios = reference_chosen_logprobs - reference_rejected_logprobs
    ```

- 上面这些代码行计算了策略模型和参考模型对于被选择样本和被拒绝样本的对数概率（对数几率）之差（这是因为 $\log\left(\frac{a}{b}\right) = \log a - \log b$）：

$$\log \left( \frac{\pi_\theta (y_w \mid x)}{\pi_\theta (y_l \mid x)} \right) \quad \text{并且} \quad \log \left( \frac{\pi_{\text{ref}}(y_w \mid x)}{\pi_{\text{ref}}(y_l \mid x)} \right)$$ 

- 接下来，代码 `logits = model_logratios - reference_logratios` 计算了模型的对数比率与参考模型的对数比率之间的差值，也就是：

$$\beta \log \left( \frac{\pi_\theta (y_w \mid x)}{\pi_{\text{ref}} (y_w \mid x)} \right)
- \beta \log \left( \frac{\pi_\theta (y_l \mid x)}{\pi_{\text{ref}} (y_l \mid x)} \right)$$ 

- 最后，`losses = -F.logsigmoid(beta * logits)` 使用对数 - sigmoid 函数来计算损失值；在原始公式中，期望符号内的项是：

$$\log \sigma \left( \beta \log \left( \frac{\pi_\theta (y_w \mid x)}{\pi_{\text{ref}} (y_w \mid x)} \right)
- \beta \log \left( \frac{\pi_\theta (y_l \mid x)}{\pi_{\text{ref}} (y_l \mid x)} \right) \right)$$ 

- 上面，我们假设对数概率已经计算好了；现在让我们定义一个 `compute_logprobs` 函数，用于计算传入上述 `compute_dpo_loss` 函数的这些对数概率，也就是 $\pi_\theta (y_w \mid x)$、${\pi_\theta (y_l \mid x)}$ 等的值： 

In [40]:
def compute_logprobs(logits, labels, selection_mask=None):
    """
    Compute log probabilities.

    Args:
      logits: Tensor of shape (batch_size, num_tokens, vocab_size)
      labels: Tensor of shape (batch_size, num_tokens)
      selection_mask: Tensor for shape (batch_size, num_tokens)

    Returns:
      mean_log_prob: Mean log probability excluding padding tokens.
    """

    # Labels are the inputs shifted by one
    labels = labels[:, 1:].clone()

    # Truncate logits to match the labels num_tokens
    logits = logits[:, :-1, :]

    log_probs = F.log_softmax(logits, dim=-1)

    # Gather the log probabilities for the actual labels
    selected_log_probs = torch.gather(
        input=log_probs,
        dim=-1,
        index=labels.unsqueeze(-1)
    ).squeeze(-1)

    if selection_mask is not None:
        mask = selection_mask[:, 1:].clone()

        # Apply the mask to filter out padding tokens
        selected_log_probs = selected_log_probs * mask

        # Calculate the average log probability excluding padding tokens
        # This averages over the tokens, so the shape is (batch_size, num_tokens)
        avg_log_prob = selected_log_probs.sum(-1) / mask.sum(-1)

        return avg_log_prob

    else:
        return selected_log_probs.mean(-1)

- 请注意，由于使用了 `torch.gather` 函数，上述函数乍一看可能有点让人望而生畏，但它与 PyTorch 的 `cross_entropy` 函数内部的实现原理非常相似。
- 例如，考虑以下示例：

In [41]:
# Sample data
logits = torch.tensor(
    [[2.0, 1.0, 0.1],
     [0.5, 2.5, 0.3]])  # Shape: (2, 3)
targets = torch.tensor([0, 2])  # Shape: (2,)


# Manual loss using torch.gather
log_softmax_logits = F.log_softmax(logits, dim=1)  # Shape: (2, 3)
selected_log_probs = torch.gather(
    input=log_softmax_logits,
    dim=1,
    index=targets.unsqueeze(1), # Shape 2, 1
).squeeze(1)  # Shape: (2,)
manual_loss = -selected_log_probs.mean()  # Averaging over the batch


# PyTorch loss
cross_entropy_loss = F.cross_entropy(logits, targets)

print(manual_loss, cross_entropy_loss)

tensor(1.4185) tensor(1.4185)


- 所以，从上面我们可以看到这两种实现方式是等价的，但让我们进一步深入探究 `torch.gather` 的工作机制。
- 考虑以下两个张量：

In [42]:
t = torch.tensor(
  [[1., 2.,],
   [3., 4.]]
)

m = torch.tensor(
  [[1, 1],
   [0, 1]]
)

- 上面，`t` 是我们要从中进行选择的张量，而 `m` 是一个掩码，用于指定我们的选择方式。
 - 例如，由于 `m` 的第一行包含 `[1, 1]`，它将两次选取 `t` 中索引位置为 `1` 的值，也就是值 `2`。
 - `m` 的第二行 `[0, 1]` 选取 `t` 第二行中索引位置为 `0` 和 `1` 的元素，即 `3.` 和 `4.`。 

In [43]:
torch.gather(input=t, dim=-1, index=m)

tensor([[2., 2.],
        [3., 4.]])

- 换句话说，`torch.gather` 是一个选择函数。
- 我们之前计算损失时，用它从包含 50,256 个词元的词表中提取与正确词元对应的对数概率。
- “正确的” 词元就是响应条目中给出的词元。 

- 关于上面的 `compute_logprobs` 函数，我们在这里使用 `torch.gather` 是因为它比 `cross_entropy` 能让我们有更多的控制权，但本质上是类似的思路。
- 我们在那里使用的 `selection_mask` 是为了选择性地忽略提示词元和填充词元。
- 然后我们可以按如下方式使用 `compute_logprobs` 函数来计算 `compute_dpo_loss` 损失函数的输入。 

In [44]:
def compute_dpo_loss_batch(batch, policy_model, reference_model, beta):
    """Compute the DPO loss on an input batch"""

    # where policy_model(batch["chosen"]) are the logits
    policy_chosen_log_probas = compute_logprobs(
        logits=policy_model(batch["chosen"]),
        labels=batch["chosen"],
        selection_mask=batch["chosen_mask"]
    )
    policy_rejected_log_probas = compute_logprobs(
        logits=policy_model(batch["rejected"]),
        labels=batch["rejected"],
        selection_mask=batch["rejected_mask"]
    )
    
    with torch.no_grad():
        ref_chosen_log_probas = compute_logprobs(
            logits=reference_model(batch["chosen"]),
            labels=batch["chosen"],
            selection_mask=batch["chosen_mask"]
        )
        ref_rejected_log_probas = compute_logprobs(
            logits=reference_model(batch["rejected"]),
            labels=batch["rejected"],
            selection_mask=batch["rejected_mask"]
        )
    loss, chosen_rewards, rejected_rewards = compute_dpo_loss(
        model_chosen_logprobs=policy_chosen_log_probas,
        model_rejected_logprobs=policy_rejected_log_probas,
        reference_chosen_logprobs=ref_chosen_log_probas,
        reference_rejected_logprobs=ref_rejected_log_probas,
        beta=beta
    )
    return loss, chosen_rewards, rejected_rewards

- 上述函数适用于单个批次，例如：

In [45]:
with torch.no_grad():
    loss = compute_dpo_loss_batch(batch, policy_model, reference_model, beta=0.1)
print(loss)

(tensor(0.6931, device='mps:0'), tensor(0., device='mps:0'), tensor(0., device='mps:0'))


- 下面，我们对这个函数进行扩展，使其能够处理数据加载器中指定的 `num_batches` 个批次：

In [46]:
def compute_dpo_loss_loader(data_loader, policy_model, reference_model, beta, num_batches=None):
    """Apply compute_dpo_loss_batch to a whole data loader"""

    total_loss, total_chosen_rewards, total_rejected_rewards = 0., 0., 0.
    if len(data_loader) == 0:
        return float("nan")

    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        # Reduce the number of batches to match the total number of batches in the data loader
        # if num_batches exceeds the number of batches in the data loader
        num_batches = min(num_batches, len(data_loader))
    for i, batch in enumerate(data_loader):
        if i < num_batches:
            loss, chosen_rewards, rejected_rewards = compute_dpo_loss_batch(
                batch=batch,
                policy_model=policy_model,
                reference_model=reference_model,
                beta=beta
            )
            total_loss += loss.item()
            total_chosen_rewards += chosen_rewards.item()
            total_rejected_rewards += rejected_rewards.item()

        else:
            break

    # calculate average
    total_loss /= num_batches
    total_chosen_rewards /= num_batches
    total_rejected_rewards /= num_batches
    return total_loss, total_chosen_rewards, total_rejected_rewards

- 为什么要指定 `num_batches` 呢？这纯粹是出于效率方面的考虑（因为每次都在整个数据集上计算损失会显著减慢训练速度）。

- 最后，我们为后续的训练函数定义了一个便捷函数；这个 `evaluate_dpo_loss_loader` 函数会计算训练数据加载器和验证数据加载器的DPO损失以及奖励，以便进行记录： 

In [47]:
def evaluate_dpo_loss_loader(policy_model, reference_model, train_loader, val_loader, beta, eval_iter):
    """Compute the DPO loss for the training and validation dataset"""

    policy_model.eval()
    with torch.no_grad():
        train_loss, train_chosen_rewards, train_rejected_rewards = compute_dpo_loss_loader(
            data_loader=train_loader,
            policy_model=policy_model,
            reference_model=reference_model,
            beta=beta,
            num_batches=eval_iter
        )

        val_loss, val_chosen_rewards, val_rejected_rewards = compute_dpo_loss_loader(
            data_loader=val_loader,
            policy_model=policy_model,
            reference_model=reference_model,
            beta=beta,
            num_batches=eval_iter
        )

    res = {
        "train_loss": train_loss,
        "train_chosen_reward": train_chosen_rewards,
        "train_rejected_reward": train_rejected_rewards,
        "val_loss": val_loss,
        "val_chosen_reward": val_chosen_rewards,
        "val_rejected_reward": val_rejected_rewards
    }

    policy_model.train()
    return res

- 在本节中，作为一个简要回顾，我们涵盖了很多内容：
  - 流程是：通过模型计算 `logits`（对数几率）$\rightarrow$ 从对数几率计算 `compute_logprobs`（计算对数概率）$\rightarrow$ 从对数概率计算 `compute_dpo_loss`（计算DPO损失）
  - 我们有 `compute_dpo_loss_batch` 函数，它为上述过程提供了便利
  - `compute_dpo_loss_loader` 实用函数将 `compute_dpo_loss_batch` 函数应用于数据加载器
  - `evaluate_dpo_loss_loader` 函数将 `compute_dpo_loss_batch` 应用于训练集和验证集的数据加载器，以便进行记录 

&nbsp;
# 5) 训练模型

- 在上一节设置好DPO损失函数之后，我们现在终于可以训练模型了。
- 请注意，这个训练函数与我们用于预训练和指令微调的函数是同一个，只是存在一些细微差别：
 - 我们将交叉熵损失替换为了新的DPO损失函数。
 - 我们还会跟踪奖励值和奖励差值，这些在基于人类反馈的强化学习（RLHF）和DPO的情境中常用于跟踪训练进度。 

In [48]:
from previous_chapters import generate_and_print_sample


def train_model_dpo_simple(
    policy_model, reference_model, train_loader, val_loader,
    optimizer, num_epochs, beta,
    eval_freq, eval_iter, start_context, tokenizer
):

    # Initialize lists to track losses and tokens seen
    tracking = {
        "train_losses": [],
        "train_chosen_rewards": [],
        "train_rejected_rewards": [],
        "val_losses": [],
        "val_chosen_rewards": [],
        "val_rejected_rewards": [],
        "tokens_seen": []
    }
    tokens_seen, global_step = 0, -1

    # Main training loop
    for epoch in range(num_epochs):
        policy_model.train()  # Set model to training mode

        for batch_idx, batch in enumerate(train_loader):

            optimizer.zero_grad()  # Reset loss gradients from previous batch iteration

            loss, chosen_rewards, rejected_rewards = compute_dpo_loss_batch(
                batch=batch,
                policy_model=policy_model,
                reference_model=reference_model,
                beta=beta
            )

            loss.backward()  # Calculate loss gradients
            optimizer.step()  # Update model weights using loss gradients

            tokens_seen += batch["chosen"].numel()
            global_step += 1

            # Optional evaluation step
            if global_step % eval_freq == 0:
                res = evaluate_dpo_loss_loader(
                    policy_model=policy_model,
                    reference_model=reference_model,
                    train_loader=train_loader,
                    val_loader=val_loader,
                    beta=beta,
                    eval_iter=eval_iter
                )
                tracking["train_losses"].append(res["train_loss"])
                tracking["train_chosen_rewards"].append(res["train_chosen_reward"])
                tracking["train_rejected_rewards"].append(res["train_rejected_reward"])
                tracking["val_losses"].append(res["val_loss"])
                tracking["val_chosen_rewards"].append(res["val_chosen_reward"])
                tracking["val_rejected_rewards"].append(res["val_rejected_reward"])
                tracking["tokens_seen"].append(tokens_seen)
                train_reward_margin = res["train_chosen_reward"] - res["train_rejected_reward"]
                val_reward_margin = res["val_chosen_reward"] - res["val_rejected_reward"]

                print(
                    f"Ep {epoch+1} (Step {global_step:06d}): "
                    f"Train loss {res['train_loss']:.3f}, Val loss {res['val_loss']:.3f}, "
                    f"Train reward margins {train_reward_margin:.3f}, "
                    f"Val reward margins {val_reward_margin:.3f}"
                )

        # Print a sample text after each epoch
        generate_and_print_sample(
            model=model,
            tokenizer=tokenizer,
            device=loss.device,
            start_context=start_context
        )

    return tracking

- 在我们开始训练之前，让我们打印出初始的损失值和奖励值： 

In [49]:
torch.manual_seed(123) # For reproducibility due to the shuffling in the data loader

res = evaluate_dpo_loss_loader(
    policy_model=policy_model,
    reference_model=reference_model,
    train_loader=train_loader,
    val_loader=val_loader,
    beta=0.1,
    eval_iter=5
)

print("Training loss:", res["train_loss"])
print("Validation loss:", res["val_loss"])

print("Train reward margin:", res["train_chosen_reward"] - res["train_rejected_reward"])
print("Val reward margin:", res["val_chosen_reward"] - res["val_rejected_reward"])

Training loss: 0.6931471824645996
Validation loss: 0.6931471824645996
Train reward margin: 0.0
Val reward margin: 0.0


- 另外，让我们来看一下模型的一些初始输出结果（验证集中的前3个示例）： 

In [50]:
torch.manual_seed(123)


for entry in val_data[:3]:

    input_text = format_input(entry)

    token_ids = generate(
        model=model,
        idx=text_to_token_ids(input_text, tokenizer).to(device),
        max_new_tokens=256,
        context_size=BASE_CONFIG["context_length"],
        eos_id=50256
    )
    generated_text = token_ids_to_text(token_ids, tokenizer)
    response_text = (
        generated_text[len(input_text):]
        .replace("### Response:", "")
        .strip()
)

    print(input_text)
    print(f"\nCorrect response:\n>> {entry['output']}")
    print(f"\nModel response:\n>> {response_text.strip()}")
    print("\n-------------------------------------\n")

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Convert the active sentence to passive: 'The chef cooks the meal every day.'

Correct response:
>> The meal is cooked by the chef every day.

Model response:
>> The meal is cooked everyday by the chef.

-------------------------------------

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Classify an input string as either a noun or a verb.

### Input:
Dance

Correct response:
>> 'Dance' can be classified as a verb.

Model response:
>> Dance is a verb.

-------------------------------------

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Rewrite the sentence using a metaphor.

### Input:
The book is very interesting.

Correct response:
>> The book is a page-turner.

Model response:
>> The book is a book.

--------

- 在上面，我们看到了模型最初的输出结果。
- 请注意，DPO（直接偏好优化）的目标是引发一些细微的风格变化；这意味着我们希望模型生成的回复内容相似，但要稍微更礼貌一些。
- 在我们执行下面这个开始训练的代码单元之前，关于一些设置有几点需要注意：
 - 我们只将策略模型的参数传递给 `AdamW` 优化器；那是我们想要优化的模型（我们不想修改参考模型）。
 - 我们只训练1个 epoch（轮次）；这是因为DPO非常容易出现模型崩溃的情况（损失值可能会降低，但模型会开始生成无意义的文本）。
 - 在DPO中，最好使用非常小的学习率。
 - β（beta）值可以从0.1增加到0.5，以降低DPO的影响（我们在这里使用0.1是为了让结果更明显）。
 - 在A100 GPU上，训练大约需要2分钟，但在较小的L4 GPU上训练则需要4分钟；在M3款的MacBook Air上训练大约需要30分钟。 

In [51]:
import time

start_time = time.time()

torch.manual_seed(123)
optimizer = torch.optim.AdamW(policy_model.parameters(), lr=5e-6, weight_decay=0.01)

num_epochs = 1
tracking = train_model_dpo_simple(
    policy_model=policy_model,
    reference_model=reference_model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    num_epochs=num_epochs,
    beta=0.1, # value between 0.1 and 0.5
    eval_freq=5,
    eval_iter=5,
    start_context=format_input(val_data[2]),
    tokenizer=tokenizer
)

end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training completed in {execution_time_minutes:.2f} minutes.")

Ep 1 (Step 000000): Train loss 0.692, Val loss 0.693, Train reward margins 0.014, Val reward margins 0.006
Ep 1 (Step 000005): Train loss 0.691, Val loss 0.692, Train reward margins 0.040, Val reward margins 0.030


KeyboardInterrupt: 

- 正如我们从上面跟踪的结果中可以看到的，损失值降低了。
- 此外，奖励差值，也就是被选中的回复和被拒绝的回复的奖励值之间的差值有所增加，这是个好迹象。
- 让我们在下一部分更具体地研究一下这些结果。 

&nbsp;
# 6) 分析结果

- 让我们通过绘制DPO损失图来开始分析结果： 

In [ ]:
from previous_chapters import plot_losses


epochs_tensor = torch.linspace(0, num_epochs, len(tracking["train_losses"]))
plot_losses(
    epochs_seen=epochs_tensor,
    tokens_seen=tracking["tokens_seen"],
    train_losses=tracking["train_losses"],
    val_losses=tracking["val_losses"],
    label="loss"
)

- 正如我们在上面所看到的，损失值持续降低，这是一个好迹象。
- 从损失值下降的趋势来看，有人可能会想再进一步训练这个模型（也鼓励读者们尝试一下），但要注意，DPO容易出现模型崩溃的情况，即模型可能会开始生成无意义的回复内容。
- 接下来，让我们看一下奖励差值： 

In [ ]:
train_reward_margins = [i-j for i,j in zip(tracking["train_chosen_rewards"], tracking["train_rejected_rewards"])]
val_reward_margins = [i-j for i,j in zip(tracking["val_chosen_rewards"], tracking["val_rejected_rewards"])]

plot_losses(
    epochs_seen=epochs_tensor,
    tokens_seen=tracking["tokens_seen"],
    train_losses=train_reward_margins,
    val_losses=val_reward_margins,
    label="reward margins"
)

- 正如我们所见，并且正如我们所期望的那样，奖励差值增大了；这与损失曲线的变化趋势相符，是个好迹象。
- 请注意，在训练过程中，DPO损失值和奖励差值是很有价值的可跟踪指标；然而，它们并不能反映全部情况。
- 最后，也是最重要的一点，我们必须对模型的回复进行定性检查。
- 在这里，我们将查看模型的回复内容（此外，你可以像第7章那样使用一个大语言模型（LLM）来给这些回复打分）。 

In [ ]:
torch.manual_seed(123)


for entry in val_data[:3]:

    input_text = format_input(entry)

    token_ids = generate(
        model=reference_model,
        idx=text_to_token_ids(input_text, tokenizer).to(device),
        max_new_tokens=256,
        context_size=BASE_CONFIG["context_length"],
        eos_id=50256
    )
    generated_text = token_ids_to_text(token_ids, tokenizer)
    reference_response_text = (
        generated_text[len(input_text):]
        .replace("### Response:", "")
        .strip()
    )

    token_ids = generate(
        model=policy_model,
        idx=text_to_token_ids(input_text, tokenizer).to(device),
        max_new_tokens=256,
        context_size=BASE_CONFIG["context_length"],
        eos_id=50256
    )
    generated_text = token_ids_to_text(token_ids, tokenizer)
    policy_response_text = (
        generated_text[len(input_text):]
        .replace("### Response:", "")
        .strip()
    )

    print(input_text)
    print(f"\nCorrect response:\n>> {entry['output']}")
    print(f"\nReference model response:\n>> {reference_response_text.strip()}")
    print(f"\nPolicy model response:\n>> {policy_response_text.strip()}")
    print("\n-------------------------------------\n")

- 从上面参考模型和策略模型的回复中我们可以看到，经过优化的模型（即策略模型）与原始模型（即参考模型）相比，确实在风格上有了些微变化。
- 例如，“Dance can be classified as a verb.”（“Dance 可以被归类为一个动词。”）变成了“The input string 'Dance' could be classified as a verb.”（“输入字符串‘Dance’可以被归类为一个动词。”），这是一个稍微更礼貌的回复（使用“could”而不是“can”使得陈述听起来不那么绝对，更具试探性）。 

In [ ]:
torch.manual_seed(123)


for entry in test_data[:3]:

    input_text = format_input(entry)

    token_ids = generate(
        model=reference_model,
        idx=text_to_token_ids(input_text, tokenizer).to(device),
        max_new_tokens=256,
        context_size=BASE_CONFIG["context_length"],
        eos_id=50256
    )
    generated_text = token_ids_to_text(token_ids, tokenizer)
    reference_response_text = (
        generated_text[len(input_text):]
        .replace("### Response:", "")
        .strip()
    )

    token_ids = generate(
        model=policy_model,
        idx=text_to_token_ids(input_text, tokenizer).to(device),
        max_new_tokens=256,
        context_size=BASE_CONFIG["context_length"],
        eos_id=50256
    )
    generated_text = token_ids_to_text(token_ids, tokenizer)
    policy_response_text = (
        generated_text[len(input_text):]
        .replace("### Response:", "")
        .strip()
    )

    print(input_text)
    print(f"\nCorrect response:\n>> {entry['output']}")
    print(f"\nReference model response:\n>> {reference_response_text.strip()}")
    print(f"\nPolicy model response:\n>> {policy_response_text.strip()}")
    print("\n-------------------------------------\n")